# Income Logistic Regression Project

My goal for this project is to create a classifier that predicts whether someone has an income below or over 50,000 dollars.

## Get the Data

In [1]:
adult = read.csv('/Users/kiroshenouda/Desktop/COMPSCI/R/R-Course-HTML-Notes/R-for-Data-Science-and-Machine-Learning/Training Exercises/Machine Learning Projects/CSV files for ML Projects/adult_sal.csv')
head(adult)

X,age,type_employer,fnlwgt,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [2]:
summary(adult)

       X              age                 type_employer       fnlwgt       
 Min.   :    1   Min.   :17.00   Private         :22696   Min.   :  12285  
 1st Qu.: 8141   1st Qu.:28.00   Self-emp-not-inc: 2541   1st Qu.: 117827  
 Median :16281   Median :37.00   Local-gov       : 2093   Median : 178356  
 Mean   :16281   Mean   :38.58   ?               : 1836   Mean   : 189778  
 3rd Qu.:24421   3rd Qu.:48.00   State-gov       : 1298   3rd Qu.: 237051  
 Max.   :32561   Max.   :90.00   Self-emp-inc    : 1116   Max.   :1484705  
                                 (Other)         :  981                    
        education     education_num                    marital     
 HS-grad     :10501   Min.   : 1.00   Divorced             : 4443  
 Some-college: 7291   1st Qu.: 9.00   Married-AF-spouse    :   23  
 Bachelors   : 5355   Median :10.00   Married-civ-spouse   :14976  
 Masters     : 1723   Mean   :10.08   Married-spouse-absent:  418  
 Assoc-voc   : 1382   3rd Qu.:12.00   Never-married 

In [3]:
str(adult)

'data.frame':	32561 obs. of  16 variables:
 $ X            : int  1 2 3 4 5 6 7 8 9 10 ...
 $ age          : int  39 50 38 53 28 37 49 52 31 42 ...
 $ type_employer: Factor w/ 9 levels "?","Federal-gov",..: 8 7 5 5 5 5 5 7 5 5 ...
 $ fnlwgt       : int  77516 83311 215646 234721 338409 284582 160187 209642 45781 159449 ...
 $ education    : Factor w/ 16 levels "10th","11th",..: 10 10 12 2 10 13 7 12 13 10 ...
 $ education_num: int  13 13 9 7 13 14 5 9 14 13 ...
 $ marital      : Factor w/ 7 levels "Divorced","Married-AF-spouse",..: 5 3 1 3 3 3 4 3 5 3 ...
 $ occupation   : Factor w/ 15 levels "?","Adm-clerical",..: 2 5 7 7 11 5 9 5 11 5 ...
 $ relationship : Factor w/ 6 levels "Husband","Not-in-family",..: 2 1 2 1 6 6 2 1 2 1 ...
 $ race         : Factor w/ 5 levels "Amer-Indian-Eskimo",..: 5 5 5 3 3 5 3 5 5 5 ...
 $ sex          : Factor w/ 2 levels "Female","Male": 2 2 2 2 1 1 1 2 1 2 ...
 $ capital_gain : int  2174 0 0 0 0 0 0 0 14084 5178 ...
 $ capital_loss : int  0 0 0 0 0 0 0 0 

## Data Cleaning

In [4]:
table(adult$type_employer)


               ?      Federal-gov        Local-gov     Never-worked 
            1836              960             2093                7 
         Private     Self-emp-inc Self-emp-not-inc        State-gov 
           22696             1116             2541             1298 
     Without-pay 
              14 

I will clean these these columns by combining some of them together, thus reducing the number of factors for classification.

In [5]:
unemployed = function(job) {
    job = as.character(job)
    if(job == 'Never-worked' | job == 'Without-pay') {
        return('Unemployed')
    } else {
        return(job)
    }
}

In [6]:
adult$type_employer = sapply(adult$type_employer, unemployed)
table(adult$type_employer)


               ?      Federal-gov        Local-gov          Private 
            1836              960             2093            22696 
    Self-emp-inc Self-emp-not-inc        State-gov       Unemployed 
            1116             2541             1298               21 

In [7]:
selfemploy = function(job) {
    job = as.character(job)
    if(job == 'Self-emp-inc' | job == 'Self-emp-not-inc') {
        return('Self-emp')
    } else {
        return(job)
    }
}

In [8]:
adult$type_employer = sapply(adult$type_employer, selfemploy)
table(adult$type_employer)


          ? Federal-gov   Local-gov     Private    Self-emp   State-gov 
       1836         960        2093       22696        3657        1298 
 Unemployed 
         21 

In [9]:
local = function(job) {
    job = as.character(job)
    if(job == 'Local-gov' | job == 'State-gov') {
        return('SL-gov')
    } else {
        return(job)
    }
}

In [10]:
adult$type_employer = sapply(adult$type_employer, local)
table(adult$type_employer)


          ? Federal-gov     Private      SL-gov    Self-emp  Unemployed 
       1836         960       22696        3391        3657          21 

In [11]:
table(adult$marital)


             Divorced     Married-AF-spouse    Married-civ-spouse 
                 4443                    23                 14976 
Married-spouse-absent         Never-married             Separated 
                  418                 10683                  1025 
              Widowed 
                  993 

Like the employment type column, I will clean the marital column by combining some of the columns together.

In [12]:
married = function(status) {
    status = as.character(status)
    if(status == 'Married-AF-spouse' | 
       status == 'Married-civ-spouse' |
       status == 'Married-spouse-absent') {
        return('Married')
    } else if(status == 'Divorced' |
              status == 'Separated' |
              status == 'Widowed') {
        return('Not-Married')
    } else {
        return(status)
    }
}

In [13]:
adult$marital = sapply(adult$marital, married)
table(adult$marital)


      Married Never-married   Not-Married 
        15417         10683          6461 

In [14]:
table(adult$country)


                         ?                   Cambodia 
                       583                         19 
                    Canada                      China 
                       121                         75 
                  Columbia                       Cuba 
                        59                         95 
        Dominican-Republic                    Ecuador 
                        70                         28 
               El-Salvador                    England 
                       106                         90 
                    France                    Germany 
                        29                        137 
                    Greece                  Guatemala 
                        29                         64 
                     Haiti         Holand-Netherlands 
                        44                          1 
                  Honduras                       Hong 
                        13                         20 
         

I will reduce the number of countries by grouping countries of the same continent/region together.

In [15]:
Asia = c('Cambodia', 'China', 'Hong', 'India', 'Iran', 'Japan',
         'Laos', 'Philippines', 'Taiwan', 'Thailand', 'Vietnam')
North.America = c('Canada', 'Puerto-Rico', 'United-States')
Europe = c('England', 'France', 'Germany', 'Greece',
           'Holand-Netherlands', 'Hungary', 'Ireland', 'Italy',
           'Poland', 'Portugal', 'Scotland', 'Yugoslavia')
Latin.and.South.America = c('Columbia', 'Cuba',
                            'Dominican-Republic', 'Ecuador', 
                            'El-Salvador', 'Guatemala', 'Haiti', 
                            'Honduras', 'Mexico', 'Nicaragua', 
                            'Outlying-US(Guam-USVI-etc)', 'Peru', 
                            'Jamaica', 'Trinadad&Tobago')
Other = c('South')

In [16]:
region = function(country) {
    if(country %in% Asia) {
        return('Asia')
    } else if(country %in% North.America) {
        return('North.America')
    } else if(country %in% Europe) {
        return('Europe')
    } else if(country %in% Latin.and.South.America) {
        return('Latin.and.South.America')
    } else {
        return('Other')
    }
}

In [17]:
adult$country = sapply(adult$country, region)

In [18]:
table(adult$country)


                   Asia                  Europe Latin.and.South.America 
                    671                     521                    1301 
          North.America                   Other 
                  29405                     663 

Since most of the variables are still continous, I will convert them to be factors for the classification model.

In [19]:
adult$type_employer = factor(adult$type_employer)
adult$education = factor(adult$education)
adult$marital = factor(adult$marital)
adult$occupation = factor(adult$occupation)
adult$relationship = factor(adult$relationship)
adult$race = factor(adult$race)
adult$sex = factor(adult$sex)
adult$country = factor(adult$country)
adult$income = factor(adult$income)

# Missing Data

In [20]:
adult[adult == '?' | adult == ' ?'] = NA

In [21]:
table(adult$type_employer)


          ? Federal-gov     Private      SL-gov    Self-emp  Unemployed 
          0         960       22696        3391        3657          21 

In [22]:
adult = na.omit(adult)

In [23]:
adult = adult[,-1]
str(adult)

'data.frame':	30718 obs. of  15 variables:
 $ age          : int  39 50 38 53 28 37 49 52 31 42 ...
 $ type_employer: Factor w/ 6 levels "?","Federal-gov",..: 4 5 3 3 3 3 3 5 3 3 ...
 $ fnlwgt       : int  77516 83311 215646 234721 338409 284582 160187 209642 45781 159449 ...
 $ education    : Factor w/ 16 levels "10th","11th",..: 10 10 12 2 10 13 7 12 13 10 ...
 $ education_num: int  13 13 9 7 13 14 5 9 14 13 ...
 $ marital      : Factor w/ 3 levels "Married","Never-married",..: 2 1 3 1 1 1 1 1 2 1 ...
 $ occupation   : Factor w/ 15 levels "?","Adm-clerical",..: 2 5 7 7 11 5 9 5 11 5 ...
 $ relationship : Factor w/ 6 levels "Husband","Not-in-family",..: 2 1 2 1 6 6 2 1 2 1 ...
 $ race         : Factor w/ 5 levels "Amer-Indian-Eskimo",..: 5 5 5 3 3 5 3 5 5 5 ...
 $ sex          : Factor w/ 2 levels "Female","Male": 2 2 2 2 1 1 1 2 1 2 ...
 $ capital_gain : int  2174 0 0 0 0 0 0 0 14084 5178 ...
 $ capital_loss : int  0 0 0 0 0 0 0 0 0 0 ...
 $ hr_per_week  : int  40 13 40 40 40 40 16 4

In [29]:
names(adult)[names(adult) == 'country'] = 'region'

# Building a Model

In [31]:
set.seed(101)
sample = sort(sample(nrow(adult), nrow(adult) * 0.8))
train = adult[sample, ]
test = adult[-sample, ]

In [32]:
model1 = glm(income ~., family = binomial(logit), data = train)
summary(model1)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Call:
glm(formula = income ~ ., family = binomial(logit), data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.2222  -0.5147  -0.1945  -0.0002   3.6421  

Coefficients: (1 not defined because of singularities)
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -5.698e+00  4.165e-01 -13.681  < 2e-16 ***
age                            2.759e-02  1.877e-03  14.699  < 2e-16 ***
type_employerPrivate          -4.893e-01  1.042e-01  -4.696 2.65e-06 ***
type_employerSL-gov           -7.387e-01  1.173e-01  -6.297 3.03e-10 ***
type_employerSelf-emp         -7.759e-01  1.156e-01  -6.709 1.96e-11 ***
type_employerUnemployed       -1.343e+01  2.239e+02  -0.060 0.952160    
fnlwgt                         6.288e-07  1.939e-07   3.242 0.001186 ** 
education11th                  1.264e-01  2.430e-01   0.520 0.602852    
education12th                  5.674e-01  2.963e-01   1.915 0.055531 .  
education1st-4th           

In [33]:
model2 = step(model1)

Start:  AIC=15990.52
income ~ age + type_employer + fnlwgt + education + education_num + 
    marital + occupation + relationship + race + sex + capital_gain + 
    capital_loss + hr_per_week + region



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess


Step:  AIC=15990.52
income ~ age + type_employer + fnlwgt + education + marital + 
    occupation + relationship + race + sex + capital_gain + capital_loss + 
    hr_per_week + region



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

                Df Deviance   AIC
<none>                15882 15990
- race           4    15891 15991
- fnlwgt         1    15893 15999
- region         4    15907 16007
- marital        2    15935 16039
- type_employer  4    15950 16050
- sex            1    15970 16076
- age            1    16101 16207
- capital_loss   1    16102 16208
- hr_per_week    1    16155 16261
- relationship   5    16171 16269
- occupation    13    16431 16513
- education     15    16710 16788
- capital_gain   1    17316 17422


In [34]:
summary(model2)


Call:
glm(formula = income ~ age + type_employer + fnlwgt + education + 
    marital + occupation + relationship + race + sex + capital_gain + 
    capital_loss + hr_per_week + region, family = binomial(logit), 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.2222  -0.5147  -0.1945  -0.0002   3.6421  

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -5.698e+00  4.165e-01 -13.681  < 2e-16 ***
age                            2.759e-02  1.877e-03  14.699  < 2e-16 ***
type_employerPrivate          -4.893e-01  1.042e-01  -4.696 2.65e-06 ***
type_employerSL-gov           -7.387e-01  1.173e-01  -6.297 3.03e-10 ***
type_employerSelf-emp         -7.759e-01  1.156e-01  -6.709 1.96e-11 ***
type_employerUnemployed       -1.343e+01  2.239e+02  -0.060 0.952160    
fnlwgt                         6.288e-07  1.939e-07   3.242 0.001186 ** 
education11th                  1.264e-01  2.430e-01   0.

In [36]:
test$predict = predict(model1, newdata = test, type = 'response')

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”

In [37]:
table(test$income, test$predict > 0.5)

       
        FALSE TRUE
  <=50K  4218  415
  >50K    591  920

In [38]:
(4218+920)/(4218+920+591+415)

[1] 0.836263

The model has an accuracy rate of about 83.63%.

In [39]:
(4218)/(4218+415)

[1] 0.9104252

The model has a recall of about 91%.

In [40]:
(4218)/(4218+591)

[1] 0.8771054

The model has a precision of about 87.7%.

I think that this is a good model, but it can be improved by refining the variables that have little or no significance in predicting one's income, like race and region.